# Assigning Jobs to Machines - The Generalized Assignment Problem

In a GAP model, we have a set of “machines” M = {1,2,…,m} and a set of “jobs” N = {1,2,…,n} that must be assigned to machines. Each machine i has a capacity $b_i$ units of work. Each job j requires $a_{ij}$ units of work to be completed if it is scheduled on machine i. All jobs must be assigned to exactly one machine. There is a fixed cost $h_i$ of using machine i.

In [3]:
using JuMP, HiGHS

M = 1:12 # number of machines
N = 1:24 # number of jobs

# for convenience -- count jobs and machines
m = length(M)
n = length(N)

# initilize the work capacity and fixed cost of each machine as 0
b = zeros(m)
h = zeros(m)

for i in 1:m
    # randomly generate values between 5 and 25 units of capacity on each machine
    b[i] = round(20*rand() + 5,digits=2)
    # randomly generate values between 5 and 15 for fixed cost of each machine
    h[i] = round(10*rand() + 5,digits=2)
end

# initialize the work required and variable cost of each job on each machine as 0
a = zeros(m,n)
c = zeros(m,n)
for i in 1:m
    for j in 1:n
        # randomly generate values between 0 and 10 units of work for each job on each machine
        a[i,j] = round(10*rand(),digits=2)
        # randomly generate values between 0 and 5 for variable cost of each job on each machine
        c[i,j] = round(5*rand(),digits=2)
    end
end

# cases represent 3 ways of modeling the logic of only assigning jobs to machines we use
# case 1 is most constraints; case 2 uses few constraints; case 3 uses most constraints and "clever" bounds
cases = [:1,:2,:3]
for case in cases
    mod = Model(HiGHS.Optimizer)

    @variable(mod, x[1:m, 1:n], Bin) # binary variables assign jobs to machines
    @variable(mod, z[1:n], Bin) # binary variables tell us which machines to use

    # objective is to minimize cost
    @objective(mod, Min, sum(c[i,j]*x[i,j] for i in 1:m for j in 1:n)
                + sum(h[i]*z[i] for i in 1:m))
        
    # use at most b[i] units of capacity on each machine i
    @constraint(mod, capacity[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i])
    @constraint(mod, jobassign[i in 1:n], sum(x[j,i] for j in 1:m) == 1)
    
    if case == :1
        # Fixed cost logic: option 1
        @constraint(mod, logic[i in 1:m, j in 1:n], x[i,j] <= 1*z[i])
    elseif case == :2
        # Fixed cost logic: option 2
        @constraint(mod, logic[i in 1:m], sum(x[i,j] for j in 1:n) <= n*z[i])
    elseif case == :3
        # Fixed cost logic: option 3
        @constraint(mod, logic[i in 1:m], sum(a[i,j]*x[i,j] for j in 1:n) <= b[i]*z[i])
    end
    println("Time for case ", case, " ")
    @time(optimize!(mod))
end

Time for case 1 


Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
299 rows, 275 cols, 1051 nonzeros
299 rows, 275 cols, 1051 nonzeros
Objective function is integral with scale 100

Solving MIP model with:
   299 rows
   275 cols (275 binary, 0 integer, 0 implied int., 0 continuous)
   1051 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   40.25471797     105.79            61.95%        0      0      0       224     0.0s
 L       0       0         0   0.00%   49.88331844     58.69             15.01%     3591     51     21       963     1.0s
 L       0       0      

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
48 rows, 275 cols, 800 nonzeros
48 rows, 275 cols, 800 nonzeros
Objective function is integral with scale 100

Solving MIP model with:
   48 rows
   275 cols (275 binary, 0 integer, 0 implied int., 0 continuous)
   800 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 R       0       0         0   0.00%   16.05511825     114.52            85.98%        0      0      0        36     0.0s
 C       0       0         0   0.00%   34.55276548     113.46            69.55%      393     49      0       169     0.0s
 L       0       0         0  

Running HiGHS 1.4.0 [date: 1970-01-01, git hash: bcf6c0b22]
Copyright (c) 2022 ERGO-Code under MIT licence terms
Presolving model
48 rows, 275 cols, 799 nonzeros
48 rows, 272 cols, 790 nonzeros
Objective function is integral with scale 100

Solving MIP model with:
   48 rows
   272 cols (272 binary, 0 integer, 0 implied int., 0 continuous)
   790 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   0               inf                  inf        0      0      0         0     0.0s
 S       0       0         0   0.00%   0               99.12            100.00%        0      0      0         0     0.0s
 R       0       0         0   0.00%   44.84589134     98.5              54.47%        0      0      0        36     0.0s
 C       0       0         0  

It is pretty clear that the third case solves the fastest. This becomes even more obvious for larger instances. Why does this happen? It's all about the convex hull! Because of how we make use of other information in the model in case 3, the LP relaxation of case 3 is the closest to the convex hull of the IP feasible set. The closer to the convex hull we get, the better. The solver works by solving a relaxed version of the model and "closing the gap" between the LP solution and the optimal IP solution. Closer to convex hull = smaller gap to close.